# Batch architecture in Recommendation Systems

In this notebook we are going to explain how to create use the batch architecture to deploy a recommendation system solution.


## 0 Global Settings and Imports

In [1]:
import json
import requests
import numpy as np
import logging
import sqlite3

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.sar import SAR

# NOTE: MODIFY THE FILE secrets.template.py
from reco_secrets import DATABASE, TABLE_NAME

In [2]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

# Other data settings
USER_COL = "userID"
ITEM_COL = "itemID"
RATING_COL = "rating"
TIMESTAMP_COL = "timestamp"
PREDICTION_COL = "prediction"

# Train test split ratio
SPLIT_RATIO = 0.75

# Model settings
SIMILARITY_TYPE = "jaccard"
TIME_DECAY = 30 # number of days until the weight of the ratings are decayed by 1/2

SEED = 42

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(levelname)-8s %(message)s")

## 1 Data Preparation


In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data[RATING_COL] = data[RATING_COL].astype(np.float32)

data.head()

2023-11-16 16:10:06,444 DEBUG    Starting new HTTPS connection (1): files.grouplens.org:443
2023-11-16 16:10:07,282 DEBUG    https://files.grouplens.org:443 "GET /datasets/movielens/ml-100k.zip HTTP/1.1" 200 4924029
2023-11-16 16:10:07,284 INFO     Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████| 4.81k/4.81k [00:01<00:00, 4.01kKB/s]


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


## 2 Model scoring


In [4]:
model = SAR(
    col_user=USER_COL,
    col_item=ITEM_COL,
    col_rating=RATING_COL,
    col_timestamp=TIMESTAMP_COL,
    similarity_type=SIMILARITY_TYPE, 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)

In [5]:
with Timer() as train_time:
    model.fit(data)

print(f"Took {train_time.interval} seconds for training.")

2023-11-16 16:10:08,988 INFO     Collecting user affinity matrix
2023-11-16 16:10:08,993 INFO     Calculating time-decayed affinities
2023-11-16 16:10:09,053 INFO     Creating index columns
2023-11-16 16:10:09,143 INFO     Calculating normalization factors
2023-11-16 16:10:09,213 INFO     Building user affinity sparse matrix
2023-11-16 16:10:09,221 INFO     Calculating item co-occurrence


2023-11-16 16:10:10,176 INFO     Calculating item similarity
2023-11-16 16:10:10,176 INFO     Using jaccard based similarity
2023-11-16 16:10:10,322 INFO     Done training


Took 1.3620430999999371 seconds for training.


In [6]:
with Timer() as scoring_time:
    top_k = model.recommend_k_items(data, top_k=TOP_K, remove_seen=True)

print("Took {} seconds for scoring.".format(scoring_time.interval))

2023-11-16 16:10:10,338 INFO     Calculating recommendation scores
2023-11-16 16:10:10,747 INFO     Removing seen items


Took 0.4584916000001158 seconds for scoring.


In [7]:
top_k.sort_values(by='prediction', ascending=False, inplace=True)

top_k.head(50)

,userID,itemID,prediction
5230,532,69,4.665657
5231,532,172,4.645321
5232,532,423,4.643408
8460,849,204,4.628795
5233,532,174,4.619922
2140,118,195,4.619201
9190,928,174,4.594107
2141,118,183,4.583690
5234,532,385,4.578142
2142,118,89,4.577481


In [8]:
top_10_recommendations = top_k.groupby('userID').head(10)
top_10_recommendations

,userID,itemID,prediction
5230,532,69,4.665657
5231,532,172,4.645321
5232,532,423,4.643408
8460,849,204,4.628795
5233,532,174,4.619922
...,...,...,...
6775,685,245,1.496370
6776,685,307,1.482737
6777,685,313,1.475710
6778,685,294,1.466042


In [9]:
top_10_recommendations[top_10_recommendations[USER_COL] == 88]

,userID,itemID,prediction
2920,88,272,3.164058
2921,88,269,3.053628
2922,88,328,3.035761
2923,88,333,2.975832
2924,88,347,2.958047
2925,88,258,2.928192
2926,88,307,2.903853
2927,88,310,2.784516
2928,88,340,2.743175
2929,88,316,2.684410


In [10]:
# Now let's look at the results for a specific user
user_id = 54

In [11]:
items_seen = data[data[USER_COL] == user_id]
items_seen

,userID,itemID,rating,timestamp
232,54,106,3.0,880937882
336,54,595,3.0,880937813
512,54,742,5.0,880934806
806,54,302,4.0,880928519
1352,54,676,5.0,880935294
...,...,...,...,...
68542,54,634,1.0,892681013
70980,54,250,4.0,880933834
74116,54,823,2.0,880938088
78663,54,405,4.0,880934806


In [12]:
items_predicted = top_k[top_k[USER_COL] == user_id].sort_values(
    by=PREDICTION_COL, ascending=False
)
items_predicted

,userID,itemID,prediction
1300,54,300,2.784323
1301,54,294,2.601673
1302,54,248,2.548543
1303,54,286,2.458506
1304,54,282,2.436808
1305,54,271,2.433754
1306,54,293,2.368300
1307,54,315,2.367518
1308,54,222,2.357715
1309,54,301,2.354047


## 3 Batch deployment

This notebook uses [SQLite](https://docs.python.org/3/library/sqlite3.html) as a database, however, any other relational or non-relational database can be used. 

In [15]:
# Establish a connection to the PostgreSQL database
conn = sqlite3.connect(database=DATABASE)

# Create a cursor object to execute SQL queries
cur = conn.cursor()

# Drop table if it already exists
query = "DROP TABLE IF EXISTS " + TABLE_NAME + ";"
cur.execute(query)

# Create a table to store your data
create_table_query = f"""
CREATE TABLE {TABLE_NAME} (
    user_id INT PRIMARY KEY,
    item1 TEXT,
    item2 TEXT,
    item3 TEXT,
    item4 TEXT,
    item5 TEXT,
    item6 TEXT,
    item7 TEXT,
    item8 TEXT,
    item9 TEXT,
    item10 TEXT
);
"""
cur.execute(create_table_query)

# Commit the changes and close the connection
conn.commit()


In [16]:
# Create a function to prepare and return the data for insertion
def prepare_data(user_group):
    user_id = user_group.name
    recommendations = user_group['itemID'].tolist()
    recommendations.extend([None] * (TOP_K - len(recommendations)) )  # Fill empty slots with NULL
    return (user_id, *recommendations)

# Use apply and groupby to efficiently generate the insert_data list
insert_data = top_10_recommendations.groupby('userID').apply(prepare_data).tolist()

In [18]:
insert_data

[(1, 423, 433, 367, 655, 385, 403, 568, 732, 318, 357),
 (2, 181, 121, 117, 7, 405, 9, 222, 742, 15, 471),
 (3, 313, 301, 748, 269, 286, 315, 678, 750, 879, 310),
 (4, 181, 286, 302, 333, 748, 174, 56, 313, 100, 121),
 (5, 82, 195, 161, 202, 96, 568, 196, 28, 265, 71),
 (6, 172, 196, 210, 97, 234, 176, 179, 96, 655, 82),
 (7, 95, 88, 58, 209, 732, 218, 739, 239, 684, 692),
 (8, 204, 69, 161, 98, 423, 230, 168, 265, 173, 234),
 (9, 174, 98, 204, 181, 172, 56, 69, 79, 100, 195),
 (10, 204, 423, 172, 89, 79, 196, 210, 97, 202, 96),
 (11, 210, 174, 202, 172, 82, 186, 195, 161, 385, 655),
 (12, 423, 210, 79, 56, 173, 22, 176, 186, 568, 183),
 (13, 203, 496, 77, 134, 47, 31, 81, 660, 479, 133),
 (14, 69, 423, 89, 135, 79, 183, 216, 64, 234, 132),
 (15, 117, 100, 151, 597, 276, 284, 294, 174, 288, 69),
 (16, 210, 82, 176, 97, 186, 196, 173, 238, 568, 132),
 (17, 121, 50, 405, 181, 25, 127, 742, 15, 98, 56),
 (18, 655, 173, 183, 176, 96, 228, 161, 385, 203, 568),
 (19, 204, 174, 216, 69, 172, 

In [19]:
# Define the SQL statement for the bulk insert
insert_sql = f"""
    INSERT INTO {TABLE_NAME} (user_id, item1, item2, item3, item4, item5, item6, item7, item8, item9, item10)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Use executemany to insert the data in a single transaction
cur.executemany(insert_sql, insert_data)


# Commit the changes 
conn.commit()


In [20]:
# Close the database connection
cur.close()
conn.close()

In [22]:
# Define the API endpoint URL
api_url = 'http://localhost:5000/recommendations'  # Replace with your API URL

# Create a dictionary with the user_id
data = {'user_id': user_id}

# Send a POST request to the API
response = requests.post(api_url, json=data)

# Check the response
if response.status_code == 200:
    recommendations = response.json()
    print("Recommendations:")
    print(json.dumps(recommendations, indent=2))
elif response.status_code == 404:
    print("User not found.")
else:
    print("Error:", response.status_code, response.text)


2023-11-16 17:53:48,107 DEBUG    Starting new HTTP connection (1): localhost:5000
2023-11-16 17:53:48,123 DEBUG    http://localhost:5000 "POST /recommendations HTTP/1.1" 500 15231


Error: 500 <!doctype html>
<html lang=en>
  <head>
    <title>sqlite3.OperationalError: no such table: recommendations
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "ceSFwE50D6kATqIGXiwR";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>OperationalError</h1>
<div class="detail">
  <p class="errormsg">sqlite3.OperationalError: no such table: recommendations
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-139729912526096">
  <h4>File <cite class="filename">"/home/u/ana